# Getting Started with Transfer Learning Using Tensorflow Hub

### What is Transfer Learning?

**Transfer Learning** is a research problem in machine learning (ML) that focuses on storing knowledge gained while solving one problem and applying it to a different but related problem

Transfer Learning overcomes the problem of isolated laerning by applying the knowledge gained during learning one task to another different but similar task. For example, knowledge gained while learning to recognize cars could be applied when trying to recognize trucks! The intuition actually came from the humans. We humans always practice *Transfer Learning*. A person who knows how to drive a bike, finds it easy to learn how to drive a car. Similarly *Transfer Learning* helps in learning difficult tasks with less effort.

### How Does TensorFlow Hub Help Us?

**TensorFlow Hub** is a repository of pre-trained machine learning models. These models are categorised in three broad problem domains - 
- Image
- Text
- Video

There are several models, which you can quickly start using without much hassle. Visit [TensorFlow Hub](https://tfhub.dev) for more details.

## The Problem

For the demonstration of the tranfer learning technique, I will be using the case of [this](https://www.kaggle.com/c/nlp-getting-started) competition. The goal here is to build a machine learning model which is able to predict whether a tweet belongs to a real disaster or not!

## Dataset

Dataset given here is tweets from different users split into training and test set. **target** is the dependent variable, which we are interested in predicting for the test set. Let's begin with our analysis.